In [17]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hyperparameter optimization") \
  .getOrCreate()

In [18]:
data = spark.read.csv("data/hotel-reservations.csv", header=True, inferSchema=True)

In [19]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  29040
Test size:  7235


In [20]:
pipeline_stages = []

In [21]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder 

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')
    
    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec', dropLast=False)
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()
    
    pipeline_stages += [string_indexer, encoder]

StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [22]:
pipeline_stages += [StringIndexer(inputCol='booking_status', outputCol= 'booking_status_index')]

In [23]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [24]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [25]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [26]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

pipeline_stages.append(assembler)

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol='features', labelCol='booking_status_index')

pipeline_stages.append(dtc)

In [28]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=pipeline_stages)

In [ ]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
  .addGrid(dtc.maxDepth, [2, 5, 10]) \
  .addGrid(dtc.maxBins, [10, 20, 40]) \
  .addGrid(dtc.minInstancesPerNode, [1, 2, 4]) \
  .build()


evaluator = MulticlassClassificationEvaluator(
    labelCol='booking_status_index',
    predictionCol='prediction',
    metricName='accuracy'
)

tvs_model = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    seed=42
)

best_model = tvs_model.fit(train_data)

In [ ]:
predictions = best_model.transform(test_data)

In [ ]:
accuracy = evaluator.evaluate(predictions) * 100
print(f'Accuracy = {accuracy:.2f}%')

In [ ]:
for param_values, metric_value in zip(paramGrid, best_model.validationMetrics):
    for param, value in param_values.items():
        print(f'{param.name} = {value}')
    print(f'Accuracy -> {metric_value}')
